### BentoML Example : Multiple Models with JsonInput
# Titanic Survival Prediction with Xgboost and Lightgbm

This is a BentoML Demo Project demonstrating how to package and serve LightBGM model for production using BentoML.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment.

In this example, we will use scikit-learn API for both `xgboost` and `lightgbm`. In general, we can use any python model. 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings

warnings.filterwarnings("ignore")

In [2]:
import bentoml
import lightgbm as lgb
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Prepare Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [3]:
%%sh
mkdir data
curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

mkdir: data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   336k      0 --:--:-- --:--:-- --:--:--  336k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28210  100 28210    0     0   202k      0 --:--:-- --:--:-- --:--:--  202k


In [4]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
y = train_df.pop("Survived")
cols = ["Pclass", "Age", "Fare", "SibSp", "Parch"]
X_train, X_test, y_train, y_test = train_test_split(
    train_df[cols], y, test_size=0.2, random_state=42
)

# Model Training

In [6]:
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train, y_train)

LGBMClassifier()

In [7]:
xgb_model = xgb.XGBRFClassifier()
xgb_model.fit(X_train, y_train)

XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=1, gamma=0, gpu_id=-1, importance_type='gain',
                interaction_constraints='', max_delta_step=0, max_depth=6,
                min_child_weight=1, missing=nan, monotone_constraints='()',
                n_estimators=100, n_jobs=0, num_parallel_tree=100,
                objective='binary:logistic', random_state=0, reg_alpha=0,
                scale_pos_weight=1, tree_method='exact', validate_parameters=1,
                verbosity=None)

In [8]:
models = {"xgb": xgb_model, "lgb": lgb_model}

## Create BentoService for model serving
We are going to use `JsonInput` and return the data as `JSON` object. `JSON` objects are passed as a **list**.

In [9]:
%%writefile multiple_models_titanic_bento_service.py

import json

import bentoml
import lightgbm as lgb
import pandas as pd
import xgboost as xgb
from bentoml.adapters import JsonInput
from bentoml.artifact import SklearnModelArtifact


@bentoml.artifacts([SklearnModelArtifact("xgb"), SklearnModelArtifact("lgb")])
@bentoml.env(
    conda_channels=["conda-forge"],
    conda_dependencies=["lightgbm==2.3.*", "pandas==1.0.*", "xgboost==1.2.*"],
)
class TitanicSurvivalPredictionService(bentoml.BentoService):
    @bentoml.api(input=JsonInput())
    def predict(self, datain):
        # datain is a list of a json object.
        df = pd.read_json(json.dumps(datain[0]), orient="table")

        data = df[["Pclass", "Age", "Fare", "SibSp", "Parch"]]
        result = pd.DataFrame()
        result["xgb_proba"] = self.artifacts.xgb.predict_proba(data)[:, 1]
        result["lgb_proba"] = self.artifacts.lgb.predict_proba(data)[:, 1]
        # make sure to return as a list of json
        return [result.to_json(orient="table")]

Overwriting multiple_models_titanic_bento_service.py


# Save BentoML service archive

In [10]:
# 1) import the custom BentoService defined above
from multiple_models_titanic_bento_service import TitanicSurvivalPredictionService

# 2) `pack` it with required artifacts
bento_service = TitanicSurvivalPredictionService()
bento_service.pack("xgb", xgb_model)
bento_service.pack("lgb", lgb_model)

# 3) save your BentoSerivce
saved_path = bento_service.save()

[2020-08-25 13:47:58,242] INFO - BentoService bundle 'TitanicSurvivalPredictionService:20200825134757_086746' saved to: /Users/thein/bentoml/repository/TitanicSurvivalPredictionService/20200825134757_086746


## Load saved BentoService for serving


In [11]:
import json

import bentoml

bento_model = bentoml.load(saved_path)


[2020-08-25 13:47:58,780] WARNING - Module `multiple_models_titanic_bento_service` already loaded, using existing imported module.


# Containerize REST API server with Docker


The BentoService SavedBundle is structured to work as a docker build context, that can be directed used to build a docker image for API server. Simply use it as the docker build context directory:

In [12]:
!docker build -t multi_models_titanic {saved_path}

Sending build context to Docker daemon  551.4kB
Step 1/15 : FROM bentoml/model-server:0.8.5
 ---> 6639eed59dc6
Step 2/15 : COPY . /bento
 ---> 8916c6323930
Step 3/15 : WORKDIR /bento
 ---> Running in e8a222116d35
Removing intermediate container e8a222116d35
 ---> ba725f7ade92
Step 4/15 : ARG PIP_INDEX_URL=https://pypi.python.org/simple/
 ---> Running in e194a9e5087c
Removing intermediate container e194a9e5087c
 ---> bfe333f7f373
Step 5/15 : ARG PIP_TRUSTED_HOST=pypi.python.org
 ---> Running in dbf5f680e79d
Removing intermediate container dbf5f680e79d
 ---> 0c9fc4ae7489
Step 6/15 : ENV PIP_INDEX_URL $PIP_INDEX_URL
 ---> Running in cb44a811259f
Removing intermediate container cb44a811259f
 ---> 7a6e44b43f45
Step 7/15 : ENV PIP_TRUSTED_HOST $PIP_TRUSTED_HOST
 ---> Running in bca323581f0b
Removing intermediate container bca323581f0b
 ---> 8f1f8b11a2d2
Step 8/15 : RUN chmod +x /bento/bentoml-init.sh
 ---> Running in 3ddc42cb25c2
Removing intermediate container 3ddc42cb25c2
 ---> 9b890a10990

Next, you can docker push the image to your choice of registry for deployment, or run it locally for development and testing:

In [13]:
# port forward to 7000 
!docker run -d -p 7000:5000 --name multi_models_titanic multi_models_titanic --enable-microbatch

docker: Error response from daemon: Conflict. The container name "/multi_models_titanic" is already in use by container "c02d33542a30598a3fc3208f5a1eacfaa3468dc0269831b38a401c9f2ca3891d". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [14]:
#check docker log to make sure there are no errors.
!docker logs multi_models_titanic

[2020-08-25 02:14:10,050] INFO - Starting BentoML API server in production mode..
[2020-08-25 02:14:10,640] INFO - get_gunicorn_num_of_workers: 4, calculated by cpu count
[2020-08-25 02:14:10,653] INFO - Running micro batch service on :5000
[2020-08-25 02:14:10 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-08-25 02:14:10 +0000] [8] [INFO] Starting gunicorn 20.0.4
[2020-08-25 02:14:10 +0000] [8] [INFO] Listening at: http://0.0.0.0:5000 (8)
[2020-08-25 02:14:10 +0000] [8] [INFO] Using worker: aiohttp.worker.GunicornWebWorker
[2020-08-25 02:14:10 +0000] [1] [INFO] Listening at: http://0.0.0.0:60115 (1)
[2020-08-25 02:14:10 +0000] [1] [INFO] Using worker: sync
[2020-08-25 02:14:10 +0000] [9] [INFO] Booting worker with pid: 9
[2020-08-25 02:14:10 +0000] [10] [INFO] Booting worker with pid: 10
[2020-08-25 02:14:10,727] INFO - Micro batch enabled for API `predict`
[2020-08-25 02:14:10,727] INFO - Your system nofile limit is 1048576, which means each instance of microbatch service is able t

# Test your running docker container

In [15]:
import requests

In [16]:
# Set the content type in the request headers
scoring_uri = "http://localhost:7000/predict"
headers = {"Content-Type": "application/json"}

In [17]:
sample_data = X_test.to_json(orient="table")

In [18]:
result = requests.post(scoring_uri, sample_data, headers=headers)

In [19]:
# check your status
result.status_code

200

In [20]:
# convert your prediction back to json
pd.read_json(result.json(), orient="table").head()

,xgb_proba,lgb_proba
0,0.294599,0.380532
1,0.389786,0.381437
2,0.263238,0.245288
3,0.801193,0.832492
4,0.311078,0.454472
